In [35]:
import numpy as np
import pandas as pd
import os

In [5]:
# Read patient data
df = pd.read_csv("../Data/subject-info-cleaned.csv")
df.head()

,Unnamed: 0,Patient ID,Follow-up period from enrollment (days),days_4years,Exit of the study,Cause of death,Age,Gender (male=1),Weight (kg),Height (cm),...,Angiotensin-II receptor blocker (yes=1),Anticoagulants/antitrombotics (yes=1),Betablockers (yes=1),Digoxin (yes=1),Loop diuretics (yes=1),Spironolactone (yes=1),Statins (yes=1),Hidralazina (yes=1),ACE inhibitor (yes=1),Nitrovasodilator (yes=1)
0,1,P0002,2045,1460,0,0,58,1,74,160,...,1,1,1,0,0,0,1,0,0,0
1,2,P0004,2044,1460,0,0,56,0,84,165,...,1,1,1,0,1,1,0,0,0,0
2,3,P0006,2043,1460,0,0,70,1,83,165,...,0,1,1,0,1,1,0,0,1,1
3,4,P0007,2042,1460,0,0,52,1,71,162,...,0,1,1,0,0,0,1,0,0,0
4,5,P0009,2039,1460,0,0,64,0,68,155,...,0,1,1,0,1,0,1,0,1,1


In [8]:
# Select columns that may go in impression section of ECG report (summary)
# Limit to categorical variables
impressions = ["Ventricular Extrasystole", "Ventricular Tachycardia", "Non-sustained ventricular tachycardia (CH>10)", 
               "Paroxysmal supraventricular tachyarrhythmia", "Bradycardia"]
df[impressions]

,Ventricular Extrasystole,Ventricular Tachycardia,Non-sustained ventricular tachycardia (CH>10),Paroxysmal supraventricular tachyarrhythmia,Bradycardia
0,1.0,0.0,0,0.0,0.0
1,1.0,0.0,0,1.0,0.0
2,2.0,0.0,0,0.0,3.0
3,1.0,1.0,0,0.0,0.0
4,1.0,0.0,0,0.0,0.0
...,...,...,...,...,...
599,2.0,1.0,0,0.0,0.0
600,2.0,0.0,0,0.0,0.0
601,2.0,0.0,0,0.0,NaN
602,1.0,0.0,0,1.0,0.0


In [21]:
# Create ECG reports
class ECGReport:
    def __init__(self, ventricular_extrasystole, ventricular_tachycardia, non_sustained_ventricular_tachycardia, paroxysmal_supraventricular_tachyarrhythmia, bradycardia):
        self.ventricular_extrasystole = ventricular_extrasystole
        self.ventricular_tachycardia = ventricular_tachycardia
        self.non_sustained_ventricular_tachycardia = non_sustained_ventricular_tachycardia
        self.paroxysmal_supraventricular_tachyarrhythmia = paroxysmal_supraventricular_tachyarrhythmia
        self.bradycardia = bradycardia
        
    def interpret_ventricular_extrasystole(self):
        ventricular_extrasystole_dict = {
            0: "No",
            1: "Monomorphic",
            2: "Polymorphic",
            3: "Couplets"
        }
        return ventricular_extrasystole_dict.get(self.ventricular_extrasystole, "Unknown ventricular extrasystole code")
    
    def interpret_ventricular_tachycardia(self):
        ventricular_tachycardia_dict = {
            0: "No",
            1: "Non-sustained VT",
            2: "Sustained VT", 
            3: "Torsade de Points"
        }
        return ventricular_tachycardia_dict.get(self.ventricular_tachycardia, "Unknown ventricular tachycardia code")
    
    def interpret_non_sustained_ventricular_tachycardia(self):
        non_sustained_ventricular_tachycardia_dict = {
            0: "No",
            1: "Yes"
        }
        return non_sustained_ventricular_tachycardia_dict.get(self.non_sustained_ventricular_tachycardia, "Unknown non sustained ventricular tachycardia code")
    
    def interpret_paroxysmal_supraventricular_tachyarrhythmia(self):
        paroxysmal_supraventricular_tachyarrhythmia_dict = {
            0: "No", 
            1: "TPSV", 
            2: "Parosysmal AF", 
            3: "Paroxismal flutter", 
            4: "Others"
        }
        return paroxysmal_supraventricular_tachyarrhythmia_dict.get(self.paroxysmal_supraventricular_tachyarrhythmia, "Unknown paroxysmal supraventricular tachyarrhythmia code")
    
    def interpret_bradycardia(self):
        bradycardia_dict = {
            0: "No",
            1: "Sinus Node Dysfunction", 
            2: "First-degree Atrioventricular block (AVB)",
            3: "Second-degree AVB - type I",
            4: "Second-degree AVB - type II", 
            5: "Third-degree AVB", 
            6: "Paroxysmal AVB"
        }
        return bradycardia_dict.get(self.bradycardia, "Unknown bradycardia code")
    
    def generate_report(self):
        return f"""
            ECG Impression:
            - Ventricular Extrasystole: {self.interpret_ventricular_extrasystole()}
            - Ventricular Tachycardia: {self.interpret_ventricular_tachycardia()}
            - Non-sustained ventricular tachycardia (CH>10): {self.interpret_non_sustained_ventricular_tachycardia()}
            - Paroxysmal supraventricular tachyarrhythmia: {self.interpret_paroxysmal_supraventricular_tachyarrhythmia()}
            - Bradycardia: {self.interpret_bradycardia()}
            """

In [ ]:
# Generate ECG impressions for all patients
df['ECG_impressions'] = df.apply(lambda row: ECGReport(
    row["Ventricular Extrasystole"], row["Ventricular Tachycardia"], row["Non-sustained ventricular tachycardia (CH>10)"], row["Paroxysmal supraventricular tachyarrhythmia"], row["Bradycardia"]).generate_report(), axis = 1)

'\n            ECG Impression:\n            - Ventricular Extrasystole: Polymorphic\n            - Ventricular Tachycardia: No\n            - Non-sustained ventricular tachycardia (CH>10): No\n            - Paroxysmal supraventricular tachyarrhythmia: No\n            - Bradycardia: Unknown bradycardia code\n            '

In [40]:
# Save to csv
directory = '/u/sswee/LLM-ECG-Domain-Adaptation/Data'
filename = 'subject-info-cleaned-with-ECGReport.csv'
full_path = os.path.join(directory, filename)
df.to_csv(full_path, index = False)